In [1]:
just_checking_integrity=False

In [2]:
import gc
import pandas as pd
import scipy.sparse as sps
import numpy as np

In [3]:
URM_test=sps.load_npz("../dataset/processed_data/URM_test_final.npz")

In [4]:
URM_test.shape

(4439967, 23690)

In [5]:
np.ravel(URM_test.sum(axis=1)).shape

(4439967,)

In [6]:
Session_ids=np.argwhere(np.ravel(URM_test.sum(axis=1)))

In [7]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

In [8]:
is_sorted(Session_ids)

True

In [9]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"]
for file in ["UCF","ICBF","Graph","ICF"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"../dataset/candidates/traditional_recs/final/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


del df_feats
del df_max

0
loading UCF
1523259
loaded UCF
0
loading ICBF
1485572
loaded ICBF
0
loading Graph
866733
loaded Graph
0
loading ICF
1815315
loaded ICF


In [10]:
ids=np.sort(dataset["Session_Id"].unique())
map_ids=pd.DataFrame()
map_ids["Session_Id"]=ids
if just_checking_integrity:
    map_ids["Real_Id"]=Session_ids[:len(ids)]
else:
    map_ids["Real_Id"]=Session_ids


In [11]:
dataset_max=dataset_max.merge(map_ids,on="Session_Id",how="left")
del dataset_max["Session_Id"]
dataset_max.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [12]:
dataset=dataset.merge(map_ids,on="Session_Id",how="left")
del dataset["Session_Id"]
dataset.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [13]:
import ast
submission_df=pd.read_csv("../dataset/candidates/NN/GRU/final/candidates.csv")
submission_df["score"]=submission_df["score"].map(ast.literal_eval)
submission_df["code"]=submission_df["code"].map(ast.literal_eval)
submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
submission_df = submission_df.explode(['code','score', 'rank'])
submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [14]:
import ast
submission_df_tr=pd.read_csv("../dataset/candidates/NN/Transformer/final/candidates.csv")
submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [15]:
import ast
submission_df_LSTM=pd.read_csv("../dataset/candidates/NN/LSTM/final/candidates.csv")
submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [16]:
dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
del submission_df
dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
del submission_df_tr
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")


In [17]:
dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")
del dataset_max

In [18]:
dataset.fillna(0,inplace=True)

In [19]:
gc.collect()

499

In [20]:
dataset_macro_features = pd.read_csv('../dataset/processed_data/macro_feats_NN_final.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')

In [21]:
# dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [22]:
dataset.head()

,Item_ID,Score_UCF,Score_ICBF,Score_Graph,Score_ICF,Session_Id,score_gru,rank_gru,score_transformer,rank_transformer,...,Max_Score_ICF,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,11,0.035168,0.115033,0.006554,0.015254,61,0.0,0.0,0.0,0.0,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,19,0.018728,0.000000,0.000000,0.000000,61,0.0,0.0,0.0,0.0,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,23,0.038990,0.072503,0.007389,0.016038,61,0.0,0.0,0.0,0.0,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,25,0.025525,0.081094,0.004554,0.008848,61,0.0,0.0,0.0,0.0,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,69,0.056398,0.085104,0.011284,0.016860,61,0.0,0.0,0.0,0.0,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [23]:
gc.collect()

23

In [24]:
dataset=dataset.sort_values("Session_Id")

In [25]:
gc.collect()

46

In [26]:
import lightgbm

In [27]:
k=10
result = []

In [28]:
for index in range(k):
    model = lightgbm.Booster(model_file=f'../rankers/model_{index}.txt')
    result.append(model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1)))

result = np.array(result)
result

array([[-1.01853501, -2.39035005, -2.39035005, ..., -2.21535968,
        -1.29375707, -2.30652403],
       [-1.11434921, -2.09785964, -2.09785964, ..., -1.78322655,
        -0.7540295 , -2.03417333],
       [-0.17699892, -0.6638325 , -0.6638325 , ..., -0.61053087,
        -0.39583665, -0.66403631],
       ...,
       [-0.41931574, -1.34089135, -1.34089135, ..., -1.28317607,
        -0.54769753, -1.31748546],
       [-0.19644651, -1.08228232, -1.08228232, ..., -1.03475824,
        -0.3845862 , -1.02652466],
       [-1.23655138, -3.86722216, -3.86722216, ..., -3.3031912 ,
        -2.12299358, -3.3206103 ]])

In [29]:
dataset["score"]=np.mean(result ,axis=0)
# dataset["score"]=np.median(result ,axis=0)

In [30]:
dataset=dataset[["Session_Id","Item_ID","score"]]
gc.collect()

130

In [31]:
dataset.head()

,Session_Id,Item_ID,score
0,61,11,-0.710325
2879180,61,4057,-1.781771
2879179,61,2498,-1.781771
2879178,61,2467,-1.781771
2879177,61,4641,-1.781771


In [32]:
dataset=dataset.groupby(['Session_Id']).agg(list).reset_index()

In [33]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [34]:
dataset["indices"]=dataset["score"].apply(lambda x: get_indices(x))

In [35]:
dataset.iloc[0]

Session_Id                                                   61
Item_ID       [11, 4057, 2498, 2467, 4641, 4157, 643, 1036, ...
score         [-0.7103247014532529, -1.7817706794946289, -1....
indices       [674, 523, 629, 617, 499, 450, 639, 642, 573, ...
Name: 0, dtype: object

In [36]:
dataset.iloc[0]["Item_ID"][536]

3424

In [37]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [38]:
dataset["predictions"]=dataset[["Item_ID","indices"]].apply(get_items,axis=1)

In [39]:
dataset=dataset[["Session_Id","predictions"]]
dataset["rank"]=dataset.apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

In [40]:
dataset

,Session_Id,predictions,rank
0,61,"[1045, 3586, 2995, 2468, 4083, 4574, 3251, 329...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"[2346, 4587, 2370, 4347, 2995, 3586, 3283, 411...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"[4148, 2704, 4146, 3326, 4162, 3827, 2330, 193...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"[2769, 16, 3338, 3026, 4202, 1516, 1139, 2180,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"[3599, 2899, 3555, 3691, 2268, 465, 2471, 3441...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
4995,443114,"[4806, 143, 2995, 4574, 89, 4083, 1369, 3290, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4996,443263,"[439, 2448, 70, 3186, 3356, 2424, 16, 989, 256...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4997,443407,"[1516, 3026, 3871, 2170, 698, 1609, 1973, 1883...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4998,443440,"[1876, 3863, 1671, 3237, 3184, 1634, 3550, 259...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [41]:
dataset["predictions"]=dataset["predictions"].apply(tuple)

In [42]:
dataset=dataset[["Session_Id","predictions","rank"]]

In [43]:
dataset.columns=["session_id","code","rank"]

In [44]:
dataset.head()

,session_id,code,rank
0,61,"(1045, 3586, 2995, 2468, 4083, 4574, 3251, 329...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"(2346, 4587, 2370, 4347, 2995, 3586, 3283, 411...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"(4148, 2704, 4146, 3326, 4162, 3827, 2330, 193...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"(2769, 16, 3338, 3026, 4202, 1516, 1139, 2180,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"(3599, 2899, 3555, 3691, 2268, 465, 2471, 3441...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [45]:
submission_df = dataset.explode(['code', 'rank']).astype(int)

In [46]:
mapping= pd.read_csv('../dataset/processed_data/map.csv')
submission_df=submission_df.merge(mapping,on="code",how="left")
del submission_df["code"]
submission_df=submission_df[["session_id","item_id","rank"]]

In [47]:
print(len(submission_df))

500000


In [48]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   session_id  500000 non-null  int32
 1   item_id     500000 non-null  int64
 2   rank        500000 non-null  int32
dtypes: int32(2), int64(1)
memory usage: 11.4 MB


In [49]:
submission_df.isna().sum()

session_id    0
item_id       0
rank          0
dtype: int64

In [50]:
submission_df.to_csv(
    f'../final_sub.csv',
    index=False
    )